# 🎥 Real-Time Sign Language Detection

This notebook demonstrates real-time sign language translation using your webcam and the trained model.

## Objectives
- Load trained model and preprocessing tools
- Initialize webcam and MediaPipe
- Perform real-time hand detection and classification
- Display predictions with confidence scores
- Test model in real-world conditions

---

## 1. Import Libraries

In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import pickle
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlow
import tensorflow as tf
from tensorflow import keras

print("✅ Libraries imported successfully")

## 2. Load Model and Preprocessing Tools

In [ ]:
# Load trained model
MODELS_DIR = 'models/saved_models'
model = keras.models.load_model(os.path.join(MODELS_DIR, 'best_model.keras'))

print("✅ Model loaded successfully")
print(f"   Model: {model.name}")

In [ ]:
# Load label encoder
with open('models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load scaler
with open('models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

class_names = label_encoder.classes_

print("\n✅ Preprocessing tools loaded")
print(f"   Classes: {class_names}")
print(f"   Number of classes: {len(class_names)}")

## 3. Initialize MediaPipe

In [ ]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

print("✅ MediaPipe initialized")

## 4. Real-Time Detection Function

In [ ]:
def realtime_detection(
    model, 
    label_encoder, 
    scaler,
    confidence_threshold=0.7,
    camera_index=0
):
    """
    Run real-time sign language detection using webcam.
    
    Args:
        model: Trained Keras model
        label_encoder: Fitted LabelEncoder
        scaler: Fitted StandardScaler
        confidence_threshold: Minimum confidence for display (0-1)
        camera_index: Camera device index
    
    Controls:
        ESC - Exit
        SPACE - Pause/Resume
        S - Save screenshot
    """
    cap = cv2.VideoCapture(camera_index)
    
    # Set camera properties
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    # Initialize MediaPipe Hands
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    ) as hands:
        
        print("\n" + "="*60)
        print("REAL-TIME SIGN LANGUAGE DETECTION")
        print("="*60)
        print("Controls:")
        print("  ESC   - Exit")
        print("  SPACE - Pause/Resume")
        print("  S     - Save screenshot")
        print("="*60 + "\n")
        
        paused = False
        frame_count = 0
        fps_start_time = datetime.now()
        fps = 0
        
        # For smoothing predictions
        prediction_history = []
        history_size = 5
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("❌ Error: Cannot read from webcam")
                break
            
            frame = cv2.flip(frame, 1)  # Mirror image
            h, w, _ = frame.shape
            
            if not paused:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = hands.process(rgb_frame)
                
                # Default display
                display_text = "No hand detected"
                display_confidence = 0.0
                display_color = (0, 0, 255)  # Red
                
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        # Draw landmarks
                        mp_drawing.draw_landmarks(
                            frame,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style()
                        )
                        
                        # Extract landmarks
                        landmarks = []
                        for lm in hand_landmarks.landmark:
                            landmarks.extend([lm.x, lm.y, lm.z])
                        
                        # Preprocess and predict
                        landmarks_array = np.array(landmarks).reshape(1, -1)
                        landmarks_scaled = scaler.transform(landmarks_array)
                        
                        prediction = model.predict(landmarks_scaled, verbose=0)
                        predicted_class = np.argmax(prediction)
                        confidence = prediction[0][predicted_class]
                        
                        # Add to history for smoothing
                        prediction_history.append(predicted_class)
                        if len(prediction_history) > history_size:
                            prediction_history.pop(0)
                        
                        # Use most common prediction in history
                        if len(prediction_history) >= 3:
                            from collections import Counter
                            smoothed_prediction = Counter(prediction_history).most_common(1)[0][0]
                        else:
                            smoothed_prediction = predicted_class
                        
                        # Get class name
                        class_name = label_encoder.inverse_transform([smoothed_prediction])[0]
                        
                        # Update display if confidence is high enough
                        if confidence >= confidence_threshold:
                            display_text = f"Sign: {class_name}"
                            display_confidence = confidence
                            display_color = (0, 255, 0)  # Green
                        else:
                            display_text = f"Low confidence ({confidence*100:.1f}%)"
                            display_color = (0, 165, 255)  # Orange
                
                # Calculate FPS
                frame_count += 1
                if frame_count % 30 == 0:
                    elapsed_time = (datetime.now() - fps_start_time).total_seconds()
                    fps = 30 / elapsed_time if elapsed_time > 0 else 0
                    fps_start_time = datetime.now()
            
            # Create overlay panel
            overlay = frame.copy()
            cv2.rectangle(overlay, (0, 0), (w, 150), (0, 0, 0), -1)
            frame = cv2.addWeighted(overlay, 0.6, frame, 0.4, 0)
            
            # Display prediction
            cv2.putText(frame, display_text, (20, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.5, display_color, 3)
            
            if display_confidence > 0:
                cv2.putText(frame, f"Confidence: {display_confidence*100:.1f}%", (20, 110), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            # Display FPS
            cv2.putText(frame, f"FPS: {fps:.1f}", (w - 150, 40), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Display status
            if paused:
                cv2.putText(frame, "PAUSED", (w - 200, h - 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)
            
            # Display controls
            cv2.putText(frame, "ESC: Exit | SPACE: Pause | S: Screenshot", (20, h - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
            
            cv2.imshow('Sign Language Translator', frame)
            
            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            
            if key == 27:  # ESC
                print("\n⏹️  Detection stopped by user")
                break
            elif key == 32:  # SPACE
                paused = not paused
                print(f"\n{'⏸️  Paused' if paused else '▶️  Resumed'}")
            elif key == ord('s') or key == ord('S'):  # S
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"outputs/visualizations/screenshot_{timestamp}.png"
                cv2.imwrite(filename, frame)
                print(f"\n📸 Screenshot saved: {filename}")
    
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n✅ Detection session ended")
    print(f"   Total frames processed: {frame_count}")

## 5. Run Real-Time Detection

In [ ]:
# Configuration
CONFIDENCE_THRESHOLD = 0.7  # Adjust this value (0.0 to 1.0)
CAMERA_INDEX = 0  # Change if you have multiple cameras

print("\nConfiguration:")
print(f"  Confidence threshold: {CONFIDENCE_THRESHOLD}")
print(f"  Camera index: {CAMERA_INDEX}")
print(f"  Classes: {class_names}")
print("\nStarting real-time detection...\n")

In [ ]:
# Run detection
# Uncomment the line below to start
realtime_detection(model, label_encoder, scaler, CONFIDENCE_THRESHOLD, CAMERA_INDEX)

## 6. Tips for Better Real-Time Performance

### Improving Accuracy:
1. **Lighting**: Ensure good, even lighting on your hand
2. **Background**: Use a plain, contrasting background
3. **Hand Position**: Keep your hand centered and fully visible
4. **Gesture Consistency**: Form gestures exactly as during training
5. **Distance**: Maintain consistent distance from camera

### Adjusting Parameters:
- **Confidence Threshold**: Lower for more predictions, higher for more accuracy
- **Detection Confidence**: Adjust in MediaPipe initialization (0.5-0.9)
- **Tracking Confidence**: Adjust for smoother tracking (0.5-0.9)
- **History Size**: Increase for smoother predictions, decrease for faster response

### Troubleshooting:
- **No hand detected**: Check lighting and hand visibility
- **Wrong predictions**: Retrain with more diverse data
- **Low FPS**: Reduce frame resolution or use GPU
- **Jittery predictions**: Increase history size for smoothing

## 7. Advanced: Record Detection Session

In [ ]:
def record_detection_session(model, label_encoder, scaler, output_filename='detection_session.avi'):
    """
    Record a detection session to video file.
    """
    cap = cv2.VideoCapture(0)
    
    # Get frame properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define codec and create VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))
    
    print(f"\n🎬 Recording to: {output_filename}")
    print("Press ESC to stop recording\n")
    
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    ) as hands:
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb_frame)
            
            # Process and display (similar to realtime_detection)
            # ... (add processing code here)
            
            # Write frame to video
            out.write(frame)
            
            cv2.imshow('Recording...', frame)
            
            if cv2.waitKey(1) & 0xFF == 27:  # ESC
                break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    print(f"\n✅ Recording saved: {output_filename}")

# Uncomment to record a session
# record_detection_session(model, label_encoder, scaler, 'outputs/visualizations/demo_session.avi')

---

## 🎯 Summary

Real-time detection setup complete!

### What Was Done:
- ✅ Loaded trained model and preprocessing tools
- ✅ Initialized MediaPipe for hand tracking
- ✅ Created real-time detection function
- ✅ Added prediction smoothing
- ✅ Implemented interactive controls
- ✅ Added recording capability

### Features:
- Real-time hand landmark detection
- Sign language classification with confidence scores
- FPS monitoring
- Prediction smoothing for stability
- Screenshot capture
- Pause/resume functionality

### Next Steps:
1. Test with different lighting conditions
2. Collect more data for underperforming classes
3. Expand to more sign language gestures
4. Add sentence construction from word sequences
5. Integrate text-to-speech for accessibility
6. Deploy as a web or mobile application

---

## 🎉 Congratulations!

You've successfully built a complete real-time sign language translator!

This project demonstrates:
- Computer vision with MediaPipe
- Deep learning with TensorFlow/Keras
- Real-time inference
- End-to-end ML pipeline

**Keep improving and expanding your model!** 🚀

---